In [1]:
%load_ext autoreload
%autoreload 2


In [3]:

from typing import List, Dict
import random
from collections import Counter
from data_config import Exp4NPN



## Post process gpt4 generations

In [16]:
"""
check
- has only one period
- starts with capital letter
- contains target string (needs to all be lower case)
- check whether the word is repeated (interesting case)
"""

'\ncheck\n- has only one period\n- starts with capital letter\n- contains target string (needs to all be lower case)\n- check whether the word is repeated (interesting case)\n\nother\n- make note of pricing\n\nsubseq experiments\n- examine what substitution of the other words for the sentence does (other top fills if we double mask)\n\n'

In [17]:

from rozlib.libs.common.data.utils_jsonl import read_from_jsonl
from paper.exp4_npn.npn_utils import GPTOutput, filter_outputs

gpt_outputs: List[GPTOutput] = read_from_jsonl(Exp4NPN.npn_gpt_outputs, GPTOutput)
gpt_outputs_after = filter_outputs(gpt_outputs)



In [20]:
print(len(gpt_outputs_after))

301


In [21]:
# note does not have bad words removed yet
def count_for_stats():
    c = Counter()
    for g in gpt_outputs_after:
        c[g.prep] += 1
    print(c)
count_for_stats()



Counter({'by': 99, 'upon': 99, 'after': 82, 'to': 21})


## human acceptability judgements

In [22]:
# add ids for use with human ranking
gpt_outputs_with_id = [(idx, go) for idx, go in enumerate(gpt_outputs_after)]
entries_randomized = [e for e in gpt_outputs_with_id]
random.seed(42)
random.shuffle(entries_randomized)

print(entries_randomized[0])


(192, GPTOutput(noun=NounRep(id=15345, token='Ġjaw', str_rep=' jaw', str_rep_no_space='jaw'), prep='by', model='gpt-4-0613', output='The boxer trained intensely, preparing for the fight jaw by jaw, focusing on his biting strength and endurance.', finish_reason='stop'))


In [23]:
from typing import Tuple
import pandas as pd

"""
Write to CSV for human analysis
"""
def write_data_rows_to_csv(entries: List[Tuple[int, GPTOutput]], file_path: str) -> None:
    # Convert dataclass instances to dictionaries
    # Convert list fields to string for CSV storage
    all_dicts: List[Dict] = []
    for e in entries:
        all_dicts.append({
            "id": e[0],
            "sent": e[1].output
        })

    # Convert to DataFrame and save to CSV
    df = pd.DataFrame(all_dicts)
    print(df.head(5))
    df.to_csv(file_path, index=False)

# uncomment to generate csv for human annotation
# write_data_rows_to_csv(entries_randomized, Exp4NPN.npn_outputs_for_human_acceptability_csv)
